<a href="https://colab.research.google.com/github/onenechan/GEE_SDFturtrial/blob/main/GEE_sdf_tutorial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install google earth engine api
!python -c "from oauth2client import crypt"
!pip install earthengine-api

# Authorization
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=rciONXthngfoSaiRWQflzpMKhoW583XiXF8e6vkH6Tg&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7sCmhPRJkARMW3wGsUzxogThpKcXGmvJSpofPcO4AUpvrSnnfqPgY

Successfully saved authorization token

In [2]:
# ライブラリのインポート
from IPython.display import Image
import ee

# Google Earth Engine の初期化
ee.Initialize()

In [4]:
# データのインプット
# 地域を選択
Area1 = ee.Geometry.Rectangle(138.7,35.3,138.8,35.4)
Landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate('2019-03-01','2019-03-15').min()

In [8]:
def VisualizeBand(data,area,band,brightness_gain):
  """
  ある一つのバンドについてのモノクロ画像を作成するメソッド。
  data : ee.ImageCollection()で指定した衛星データ
  area  : ee.Geometryで得られた位置データ
  band  : バンド数の指定
  brightness_gain : 明るさを調整するためのゲイン(正の実数)
  """
  Band = data.select("B%i" %band) #bandに対応するバンドを選択
  
  # expression(式,引数の指定)を用いて計算をしている。
  # ゲインと呼ばれる定数をかけることで値を調整している。
  # 可視化する際は0とそれ以下の値を最小値(黒)、1とそれ以上の値を最大値(白)として明るさを配分しているため、
  # ゲインを変更することで各バンドの明るさを変更することができる。
  Band_gain = data.expression("Band*gain",{"Band":Band,"gain":brightness_gain})
  
  # データの範囲を選択し、画像データにする。
  clipped_image = ee.Image(Band_gain).clip(area)
  
  # データの範囲を緯度経度で表したリストを作成する。
  region_geom = area.getInfo()['coordinates']
  
  # 画像のサムネイルの作成と画像の可視化を行う。
  image_url = clipped_image.getThumbUrl({'min':0,'max':1,'dimensions':'800','region':region_geom})
  return Image(url=image_url)


In [9]:
VisualizeBand(Landsat8_collection,Area1,8,3)

In [12]:
def TrueColor(data,area,brightness_gain):
  """
  TrueColor画像の可視化を行うメソッド。
  image : ee.ImageCollection()で指定した衛星データ
  area  : ee.Geometryで得られた位置データ
  brightness_gain : 明るさを調整するためのゲイン(正の実数)
  """
  Band4 = data.select("B4") #赤色に対応するバンドを選択
  Band3 = data.select("B3") #緑色に対応するバンドを選択
  Band2 = data.select("B2") #青色に対応するバンドを選択
  
  # expression(式,引数の指定)を用いて計算をしている。
  # ゲインと呼ばれる定数をかけることで値を調整している。
  # 可視化する際は0とそれ以下の値を最小値(黒)、1とそれ以上の値を最大値(白)として明るさを配分しているため、ゲインを変更することで各バンドの明るさを変更することができる。
  red = data.expression("Red*gain_R",{"Red":Band4,"gain_R":brightness_gain})
  green = data.expression("Green*gain_G",{"Green":Band3,"gain_G":brightness_gain})
  blue = data.expression("Blue*gain_B",{"Blue":Band2,"gain_B":brightness_gain})
  
  # TrueColorのバンドを重ねたデータを作成する。
  image = ee.Image(red).addBands(green).addBands(blue)
  
  # データの範囲を指定する。
  clipped_image = image.clip(area)
  
  # データの範囲を緯度経度で表したリストを作成する。
  region_geom = area.getInfo()['coordinates']
  
  # 画像のサムネイルの作成と画像の可視化を行う。
  image_url = clipped_image.getThumbUrl({'min':0,'max':1,'dimensions':'800','region':region_geom})
  print(image_url)
  return Image(url=image_url)

In [13]:
TrueColor(Landsat8_collection,Area1,3)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/2a09fc4e63a3bbedf04c81ada0c2b014-0dcf69d68c7bb4671d526da866fc0fb2:getPixels
